<a href="https://colab.research.google.com/github/saqib-sarwar/sts/blob/main/sts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Stsbenchmark.tar.gz to Stsbenchmark.tar.gz


In [2]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.2 MB/s 
     |████████████████████████████████| 5.5 MB 58.3 MB/s 
     |████████████████████████████████| 1.3 MB 59.5 MB/s 
     |████████████████████████████████| 182 kB 72.6 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5a76d7226070b29bfe2a3707cb4c8fa54540cf0e70a6b3c1cedc88a0204668cf
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
import tarfile

#Open tar file in read mode and Extract the files into the /tmp folder
file = tarfile.open('/content/Stsbenchmark.tar.gz', 'r') 
file.extractall('/content/temp') 
file.close()

In [4]:
import pandas as pd
import numpy as np

In [5]:
train_path = "/content/temp/stsbenchmark/sts-train.csv"
dev_path = "/content/temp/stsbenchmark/sts-dev.csv"
test_path = "/content/temp/stsbenchmark/sts-test.csv"

In [6]:
# View contents of file
sts_train = pd.read_csv(train_path, sep="\t", header=None, on_bad_lines="skip")
sts_dev = pd.read_csv(dev_path, sep="\t", header=None, on_bad_lines="skip")
sts_test = pd.read_csv(test_path, sep="\t", engine='python', header=None, on_bad_lines="skip")
sts_train.head(5)

,0,1,2,3,4,5,6
0,main-captions,MSRvid,2012test,1,5.00,A plane is taking off.,An air plane is taking off.
1,main-captions,MSRvid,2012test,4,3.80,A man is playing a large flute.,A man is playing a flute.
2,main-captions,MSRvid,2012test,5,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,main-captions,MSRvid,2012test,6,2.60,Three men are playing chess.,Two men are playing chess.
4,main-captions,MSRvid,2012test,9,4.25,A man is playing the cello.,A man seated is playing the cello.


In [7]:
# Check last elements
sts_train.tail()

,0,1,2,3,4,5,6
5504,main-news,headlines,2015,1489,1.2,"Palestinian hunger striker, Israel reach deal",Palestinian activist detained in Israeli raid
5505,main-news,headlines,2015,1493,4.8,Assad says Syria will comply with UN arms reso...,Syria's Assad vows to comply with U.N. resolution
5506,main-news,headlines,2015,1496,4.6,South Korean President Sorry For Ferry Response,S. Korean president 'sorry' for ferry disaster
5507,main-news,headlines,2015,1498,0.0,Food price hikes raise concerns in Iran,American Chris Horner wins Tour of Spain
5508,main-news,headlines,2015,1499,2.0,Obama made last-minute decision on Syria approval,Obama mulls limited military action in Syria


In [8]:
sts_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5509 entries, 0 to 5508
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       5509 non-null   object 
 1   1       5509 non-null   object 
 2   2       5509 non-null   object 
 3   3       5509 non-null   int64  
 4   4       5509 non-null   float64
 5   5       5509 non-null   object 
 6   6       5506 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 301.4+ KB


In [9]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

In [10]:
batch_size = 16
epochs = 3
model_save_path = 'content/temp/model_'

In [11]:

# Load the pre-trained model
model = CrossEncoder('cross-encoder/stsb-roberta-base',  num_labels=1)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/142 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [12]:
train_data = []
dev_data = []
test_data = []

# Normalize Scores in range 0-1
train_score = [float(x/5.0) for x in sts_train[4]]
dev_score = [float(x/5.0) for x in sts_dev[4]]
test_score = [float(x/5.0) for x in sts_test[4]]

train_data = [InputExample(texts=[s1, s2], label=t_score) for s1,s2,t_score in zip(sts_train[5], sts_train[6], sts_train[4])]
#For getting symmetric scores, i.e. CrossEncoder(S1,S2) = CrossEncoder(S2,S1), we pass both combinations to the train set
train_data = train_data +  [InputExample(texts=[s2, s1], label=t_score) for s1,s2,t_score in zip(sts_train[5], sts_train[6], sts_train[4])]

dev_data = [InputExample(texts=[s1, s2], label=t_score) for s1,s2,t_score in zip(sts_dev[5], sts_dev[6], sts_dev[4])]
test_data = [InputExample(texts=[s1, s2], label=t_score) for s1,s2,t_score in zip(sts_test[5], sts_test[6], sts_test[4])]



In [13]:
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

# Define an evaluator
evaluator = CECorrelationEvaluator.from_input_examples(dev_data, name='sts-dev')

In [14]:
# Train the model
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=epochs,
          output_path=model_save_path)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/689 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
# Load model and evaluate on test set
model = CrossEncoder(model_save_path)

In [ ]:
evaluator = CECorrelationEvaluator.from_input_examples(test_data, name='sts-test')
evaluator(model)